#### Data Source

The data for this project was housed in a parent folder called Albany. This folder contained four other folders, one for each month (July, August, September and October). The names of the folders are as follows: "July_3rd_2023", "Aug_9th_2023", "Sept_2nd_2023" and "Oct_1st_2023". Each folder contained several csv files for each month. For each month, the csv provided were: "listings", "calendar","reviews" "reviews summarized" and "listings summarized". The files of interest for this project were listings and calendar. For efficiency and time, the best way to work with this data was by creating one parent csv file with all the relevant data. For each month,the listings and calendar files were merged together using 'id' column on listings file and 'listing_id' column on calendar file.So, for there was one csv file for each month. These four files were then concatenated to create one massive parent file. To achieve this, ChatGPT was engaged. See the prompt below that was used to generate the code to accomplish this task

In [1]:
''' 
I have 4 folders names as follows: "Aug_9th_2023", "July_3rd_2023","Oct_1st_2023" and "Sept_2nd_2023". 
Each folder contains about 3 to 4 csv files. I am only interested in two csv files in each folder. They're 
called "listings" and "calendar".  Listings contain a column called "id" and calendar contains a column 
called "listing_id". Here is what I want to do:

1. In each folder, merge listings and calendar csv files on listings.id = calendar.listing_id. 
I only want records that are present in both csv files (similar to inner join in SQL). If an an 
id in one file doesn't have a corresponding id in the other file,  do not return those records. 
I want this process to occur in every folder. The result should be one merged csv file per folder

2.  Afterwards, I want to concatenate the 4 merged csv files into 1 larger csv files. Drop any duplicate 
rows (rows which have exact values in all of the columns)

Write a code that would achieve this result
'''


import os
import pandas as pd

# Function to merge listings and calendar files
def merge_files(folder_path):
    listings_file = None
    calendar_file = None
    
    # Locate listings and calendar files
    for file in os.listdir(folder_path):
        if file.startswith('listings') and file.endswith('.csv'):
            listings_file = os.path.join(folder_path, file)
        elif file.startswith('calendar') and file.endswith('.csv'):
            calendar_file = os.path.join(folder_path, file)
    
    # Check if listings and calendar files are found
    if listings_file is None or calendar_file is None:
        print(f"Error: Listings or calendar file not found in folder {folder_path}")
        return None
    
    # Read listings and calendar files
    listings_df = pd.read_csv(listings_file)
    calendar_df = pd.read_csv(calendar_file)
    
    # Merge based on 'id' and 'listing_id' columns
    merged_df = pd.merge(listings_df, calendar_df, left_on='id', right_on='listing_id', how='inner')
    return merged_df

# List of folders
folders = ['Aug_9th_2023', 'July_3rd_2023', 'Oct_1st_2023', 'Sept_2nd_2023']

# List to store merged DataFrames from each folder
merged_dfs = []

# Iterate over folders
for folder in folders:
    folder_path = os.path.join('Albany/',folder)
    merged_df = merge_files(folder_path)
    if merged_df is not None:
        merged_dfs.append(merged_df)

# Concatenate DataFrames from all folders
parent_df = pd.concat(merged_dfs, ignore_index=True)

# Drop duplicate rows
parent_df.drop_duplicates(inplace=True)

# Write final DataFrame to a CSV file
parent_df.to_csv('parent_data.csv', index=False)


In [2]:
''' 
creating two copies for backup in case there are issues with the dataframe
 down the line
 '''
parent_subfile_one = parent_df.copy()
parent_subfile_two = parent_df.copy()

In [3]:
# Checking the columns available 
parent_subfile_one.columns

Index(['id', 'listing_url', 'scrape_id', 'last_scraped', 'source', 'name',
       'description', 'neighborhood_overview', 'picture_url', 'host_id',
       'host_url', 'host_name', 'host_since', 'host_location', 'host_about',
       'host_response_time', 'host_response_rate', 'host_acceptance_rate',
       'host_is_superhost', 'host_thumbnail_url', 'host_picture_url',
       'host_neighbourhood', 'host_listings_count',
       'host_total_listings_count', 'host_verifications',
       'host_has_profile_pic', 'host_identity_verified', 'neighbourhood',
       'neighbourhood_cleansed', 'neighbourhood_group_cleansed', 'latitude',
       'longitude', 'property_type', 'room_type', 'accommodates', 'bathrooms',
       'bathrooms_text', 'bedrooms', 'beds', 'amenities', 'price_x',
       'minimum_nights_x', 'maximum_nights_x', 'minimum_minimum_nights',
       'maximum_minimum_nights', 'minimum_maximum_nights',
       'maximum_maximum_nights', 'minimum_nights_avg_ntm',
       'maximum_nights_avg_ntm

In [4]:
# Creating a subset dataframe/dataset with only relevant columns
subset_df = parent_subfile_two[['id','name','date','description','host_id','host_name','host_since',
              'host_location','host_response_time','host_response_rate',
              'host_acceptance_rate','host_is_superhost','host_neighbourhood',
              'host_listings_count','host_total_listings_count','host_verifications',
              'host_has_profile_pic','host_identity_verified','property_type',
              'room_type','accommodates','bathrooms','bedrooms','beds','amenities',
              'price_x','price_y','adjusted_price','minimum_nights_x','minimum_nights_y',
              'maximum_nights_x','maximum_nights_y', 'calendar_updated','has_availability',
              'number_of_reviews','review_scores_rating','review_scores_accuracy',
              'reviews_per_month']].copy()

# Drop duplicates
subset_df = subset_df.drop_duplicates()

# Check for duplicates
duplicates_count = subset_df.duplicated().sum()
duplicates_count

# Moving forward, subset_df will be used

0

In [5]:
# Displaying top 2 rows of subset_df
subset_df.head(2)

,id,name,date,description,host_id,host_name,host_since,host_location,host_response_time,host_response_rate,...,minimum_nights_x,minimum_nights_y,maximum_nights_x,maximum_nights_y,calendar_updated,has_availability,number_of_reviews,review_scores_rating,review_scores_accuracy,reviews_per_month
0,54126420.0,Home in Albany · ★4.80 · 1 bedroom · 1 bed · 1...,8/7/2024,1 bedroom private apartment located in a quiet...,37769478.0,Erion,7/7/2015,"Albany, NY",within an hour,100%,...,1.0,1,1125.0,1125,NaN,t,55.0,4.8,4.89,3.89
1,54126420.0,Home in Albany · ★4.80 · 1 bedroom · 1 bed · 1...,8/6/2024,1 bedroom private apartment located in a quiet...,37769478.0,Erion,7/7/2015,"Albany, NY",within an hour,100%,...,1.0,1,1125.0,1125,NaN,t,55.0,4.8,4.89,3.89


In [6]:
# Checking out the shape of the dataframe
subset_df.shape

# The dataframe has 395660 rows and 38 columns

(395660, 38)

In [7]:
# Exploring general information about the dataset
subset_df.info()

'''
Quick observations show that there are some null values in some columns. 
Some columns like'id' and date have incorrect datatypes

'''

<class 'pandas.core.frame.DataFrame'>
Int64Index: 395660 entries, 0 to 395659
Data columns (total 38 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   id                         395660 non-null  float64
 1   name                       395660 non-null  object 
 2   date                       395660 non-null  object 
 3   description                387265 non-null  object 
 4   host_id                    395660 non-null  float64
 5   host_name                  395660 non-null  object 
 6   host_since                 395660 non-null  object 
 7   host_location              312075 non-null  object 
 8   host_response_time         376315 non-null  object 
 9   host_response_rate         376315 non-null  object 
 10  host_acceptance_rate       379600 non-null  object 
 11  host_is_superhost          353685 non-null  object 
 12  host_neighbourhood         296380 non-null  object 
 13  host_listings_count        39

"\nQuick observations show that there are some null values in some columns. \nSome columns like'id' and date have incorrect datatypes\n\n"

In [8]:
# Exploring null values
# Displaying all columns with null values
all_columns = subset_df.isnull().sum()
null_value_columns = all_columns[all_columns > 0]
null_value_columns.sort_values(ascending=False)

bathrooms                 395660
calendar_updated          395660
bedrooms                  114245
host_neighbourhood         99280
host_location              83585
host_is_superhost          41975
review_scores_rating       36865
review_scores_accuracy     36865
reviews_per_month          36865
host_response_time         19345
host_response_rate         19345
host_acceptance_rate       16060
description                 8395
beds                        1825
dtype: int64

In [9]:
# It appears that bathroom and calendar_updated columns are completely empty. Hence, dropping them

subset_df.drop(columns=['bathrooms','calendar_updated'],inplace=True)

In [10]:
''' Handling null values '''

# For bedrooms, replacing NaN with median value
subset_df['bedrooms'] = subset_df['bedrooms'].fillna(value=subset_df['bedrooms'].median())

# For host_neighbourhood, replacing NaN with "unknown"
subset_df['host_neighbourhood'] = subset_df['host_neighbourhood'].fillna('unknown')

# For host_location, replacing NaN with "unknown"
subset_df['host_location'] = subset_df['host_location'].fillna('unknown')

# For host_is_superhost, replacing NaN with "unknown"
subset_df['host_is_superhost'] = subset_df['host_is_superhost'].fillna('unknown')

# For review_scores_rating, replacing NaN with median value
median_value = subset_df['review_scores_rating'].median()
subset_df['review_scores_rating'] = subset_df['review_scores_rating'].fillna(median_value)

# For review_scores_accuracy, replacing NaN with median value
median_review_accuracy = subset_df['review_scores_accuracy'].median()
subset_df['review_scores_accuracy'] = subset_df['review_scores_accuracy'].fillna(median_review_accuracy)

# For reviews_per_month. replacing NaN with median value
med_rev_per_month = subset_df['reviews_per_month'].median()
subset_df['reviews_per_month'] = subset_df['reviews_per_month'].fillna(med_rev_per_month)

# For host_response_time, replacing NaN with "unknown"
subset_df['host_response_time'] = subset_df['host_response_time'].fillna('unknown')

# For host_response_rate, replacing NaN with median value
''' First removing %, changing dtype to float then replacing NaN with median value '''
subset_df['host_response_rate'] = subset_df['host_response_rate'].str.replace('%','').astype(float)
med_resp_rate = subset_df['host_response_rate'].median()
subset_df['host_response_rate'] = subset_df['host_response_rate'].fillna(med_resp_rate)

# For host_acceptance_rate, replacing NaN with median value: Same procedure as host_response_rate
subset_df['host_acceptance_rate'] = subset_df['host_acceptance_rate'].str.replace('%','').astype(float)
med_acc_rate = subset_df['host_acceptance_rate'].median()
subset_df['host_acceptance_rate'] = subset_df['host_acceptance_rate'].fillna(med_acc_rate)

# For description, replacing NaN with "unknown"
subset_df['description'] = subset_df['description'].fillna('unknown')

# For beds, replacing NaN with median value
med_beds = subset_df['beds'].median()
subset_df['beds'] = subset_df['beds'].fillna(med_beds)



In [11]:
# Checking to see if all null_values have been taken care of
subset_df.isnull().sum()

# All the null values have been handled

id                           0
name                         0
date                         0
description                  0
host_id                      0
host_name                    0
host_since                   0
host_location                0
host_response_time           0
host_response_rate           0
host_acceptance_rate         0
host_is_superhost            0
host_neighbourhood           0
host_listings_count          0
host_total_listings_count    0
host_verifications           0
host_has_profile_pic         0
host_identity_verified       0
property_type                0
room_type                    0
accommodates                 0
bedrooms                     0
beds                         0
amenities                    0
price_x                      0
price_y                      0
adjusted_price               0
minimum_nights_x             0
minimum_nights_y             0
maximum_nights_x             0
maximum_nights_y             0
has_availability             0
number_o

In [12]:
# Checking for incorrect datatypes
subset_df.dtypes

id                           float64
name                          object
date                          object
description                   object
host_id                      float64
host_name                     object
host_since                    object
host_location                 object
host_response_time            object
host_response_rate           float64
host_acceptance_rate         float64
host_is_superhost             object
host_neighbourhood            object
host_listings_count          float64
host_total_listings_count    float64
host_verifications            object
host_has_profile_pic          object
host_identity_verified        object
property_type                 object
room_type                     object
accommodates                 float64
bedrooms                     float64
beds                         float64
amenities                     object
price_x                       object
price_y                       object
adjusted_price                object
m

In [13]:
''' The following columns need to be changed datatypes
 float to int: beds, bedrooms, number_of_reviews, host_listings_count,
               host_total_listings_count, reviews_per_month, minimum and maximum nights
 float to object: id, host_id
 object to float: price_x, price_y, adjusted_price
 object to date: date, host_since
 '''

' The following columns need to be changed datatypes\n float to int: beds, bedrooms, number_of_reviews, host_listings_count,\n               host_total_listings_count, reviews_per_month, minimum and maximum nights\n float to object: id, host_id\n object to float: price_x, price_y, adjusted_price\n object to date: date, host_since\n '

In [14]:
# Converting float to int
subset_df = subset_df.astype({'beds': 'int','bedrooms':'int','number_of_reviews':'int',
                             'host_listings_count':'int','host_total_listings_count':'int',
                             'reviews_per_month':'int','minimum_nights_x':'int',
                             'maximum_nights_x':'int'})

# Converting float to object
subset_df = subset_df.astype({'id':'object','host_id':'object'})

# Converting object to float
# This code removes the dollar sign and any commas first then converting it to float
price_columns = ['price_x','price_y','adjusted_price']
subset_df[price_columns] = subset_df[price_columns].replace('[\$,]', '', regex=True).astype(float)

# Converting object to date
date_columns = ['date','host_since']
subset_df[date_columns] = subset_df[date_columns].apply(pd.to_datetime, format='%m/%d/%Y')

In [15]:
# Checking to make sure the datatypes have converted correctly
subset_df.dtypes

id                                   object
name                                 object
date                         datetime64[ns]
description                          object
host_id                              object
host_name                            object
host_since                   datetime64[ns]
host_location                        object
host_response_time                   object
host_response_rate                  float64
host_acceptance_rate                float64
host_is_superhost                    object
host_neighbourhood                   object
host_listings_count                   int32
host_total_listings_count             int32
host_verifications                   object
host_has_profile_pic                 object
host_identity_verified               object
property_type                        object
room_type                            object
accommodates                        float64
bedrooms                              int32
beds                            

In [16]:
# Renaming 'id' column to 'listing_id' and 'name' to 'listing_name'
subset_df.rename(columns={'id':'listing_id','name':'listing_name'}, inplace=True)

#Checking to see whether the renaming was successful
subset_df[['listing_id','listing_name']].head()

,listing_id,listing_name
0,54126420.0,Home in Albany · ★4.80 · 1 bedroom · 1 bed · 1...
1,54126420.0,Home in Albany · ★4.80 · 1 bedroom · 1 bed · 1...
2,54126420.0,Home in Albany · ★4.80 · 1 bedroom · 1 bed · 1...
3,54126420.0,Home in Albany · ★4.80 · 1 bedroom · 1 bed · 1...
4,54126420.0,Home in Albany · ★4.80 · 1 bedroom · 1 bed · 1...


In [17]:
'''Exploring the listing_name column'''
subset_df.listing_name.unique()

''' 
This column appears to contain 
     - Listing name
     - Star rating
     - # bedrooms
     - # beds
     - # bath
'''

' \nThis column appears to contain \n     - Listing name\n     - Star rating\n     - # bedrooms\n     - # beds\n     - # bath\n'

In [24]:
# Creating new columns:
new_columns = ['property_name', 'star_rating', 'n_bedrooms', 'n_beds', 'n_baths']
subset_df[new_columns] = subset_df['listing_name'].str.split(' · ', expand=True)

# Removing stars from the 'star_rating' column
subset_df['star_rating'] = subset_df['star_rating'].str.replace('★', '')

# Checking to see whether the listing_name column has been split and new ones have been added
subset_df.columns


Index(['listing_id', 'listing_name', 'date', 'description', 'host_id',
       'host_name', 'host_since', 'host_location', 'host_response_time',
       'host_response_rate', 'host_acceptance_rate', 'host_is_superhost',
       'host_neighbourhood', 'host_listings_count',
       'host_total_listings_count', 'host_verifications',
       'host_has_profile_pic', 'host_identity_verified', 'property_type',
       'room_type', 'accommodates', 'bedrooms', 'beds', 'amenities', 'price_x',
       'price_y', 'adjusted_price', 'minimum_nights_x', 'minimum_nights_y',
       'maximum_nights_x', 'maximum_nights_y', 'has_availability',
       'number_of_reviews', 'review_scores_rating', 'review_scores_accuracy',
       'reviews_per_month', 'property_name', 'star_rating', 'n_bedrooms',
       'n_beds', 'n_baths'],
      dtype='object')

In [37]:
subset_df[['host_listings_count','host_total_listings_count']]

,host_listings_count,host_total_listings_count
0,8,8
1,8,8
2,8,8
3,8,8
4,8,8
...,...,...
395655,8,8
395656,8,8
395657,8,8
395658,8,8


In [40]:
'''
Exploring the columns 'host_listings_count' and 'host_total_listings_count' to see whether 
they're duplicate of one another. If yes, one will be dropped. If not, both would be kept and
further clarification would be required to explain the difference between them
'''
# Checking if the two columns are duplicates of each other
are_duplicates = subset_df['host_listings_count'].equals(subset_df['host_total_listings_count'])

# Printing the result
print("Are host_listings_count and host_total_listings_count duplicates?", are_duplicates)

# The result shows that the columns are not duplicate.

Are host_listings_count and host_total_listings_count duplicates? False


In [45]:
#Identifying instances in which the two columns dont have same values
# Filtering rows where values in host_listings_count and host_total_listings_count are different
differences = subset_df[subset_df['host_listings_count'] != subset_df['host_total_listings_count']]

# Displaying the rows in both columns where differences occur
differences[['host_listings_count','host_total_listings_count']]

# Oer 180k rows, roughly 46% have different values for both columns. Keeping both


,host_listings_count,host_total_listings_count
1095,1,3
1096,1,3
1097,1,3
1098,1,3
1099,1,3
...,...,...
395290,1,3
395291,1,3
395292,1,3
395293,1,3


In [83]:
'''
Exploring the columns 'minimum_nights_x' and 'minimum_nights_y' to see whether 
they're duplicate of one another. If yes, one will be dropped. If not, both would be kept and
further clarification would be required to explain the difference between them
'''
# Checking if the two columns are duplicates of each other
nights_duplicated = subset_df['minimum_nights_x'].equals(subset_df['minimum_nights_y'])

# Printing the result
print('Is minimum_nights_x and minimum_nights_y duplicated?', nights_duplicated)

# Filtering rows to display only the ones with mismatching values, then extracting the two columns
subset_df[subset_df['minimum_nights_x'] != subset_df['minimum_nights_y']][['minimum_nights_x',
                                                                           'minimum_nights_y']]

'''
Only 24215 out of 395660 rows, roughly 6% are a mismatch between minimum_nights_x and minimum_nights_y
These two columns will be averaged
'''


Is minimum_nights_x and minimum_nights_y duplicated? False


,minimum_nights_x,minimum_nights_y
2194,1,2
2201,1,2
2208,1,2
2215,1,2
2222,1,2
...,...,...
395280,2,3
395281,2,3
395282,2,3
395283,2,3


In [74]:
# Creating a column with average values for 'minimum_nights_x' and 'minimum_nights_y'
subset_df['avg_minimum_nights'] = subset_df[['minimum_nights_x','minimum_nights_y']].mean(axis=1)
subset_df['avg_minimum_nights'] = subset_df['avg_minimum_nights'].astype(int)

# Confirming that the column creation has executed
subset_df['avg_minimum_nights']


0         1
1         1
2         1
3         1
4         1
         ..
395655    7
395656    7
395657    7
395658    7
395659    7
Name: avg_minimum_nights, Length: 395660, dtype: int32

In [81]:
'''
Exploring the columns 'maximum_nights_x' and 'maximum_nights_y' to see whether 
they're duplicate of one another. If yes, one will be dropped. If not, both would be kept and
further clarification would be required to explain the difference between them
'''

max_nights_duplicated = subset_df['maximum_nights_x'].equals(subset_df['maximum_nights_y'])
print('Is maximum_nights_x and maximum_nights_y duplicated?', max_nights_duplicated)

subset_df[subset_df['maximum_nights_x'] != subset_df['maximum_nights_y']][['maximum_nights_x',
                                                                           'maximum_nights_y']]

'''
91852 out of 395660 rows, roughly 23% are a mismatch between minimum_nights_x and minimum_nights_y
This mismatch is above 10% convention. The columns will be averaged, but it's worth noting that
the average value is likely to result into a misleading analyis.
'''

Is maximum_nights_x and maximum_nights_y duplicated? False


,maximum_nights_x,maximum_nights_y
2190,30,1125
2191,30,1125
2192,30,1125
2193,30,1125
2194,30,1125
...,...,...
394925,365,1125
394926,365,1125
394927,365,1125
394928,365,1125


In [79]:
# Creating a column with average values for 'maximum_nights_x' and 'maximum_nights_y'
subset_df['avg_maximum_nights'] = subset_df[['maximum_nights_x','maximum_nights_y']].mean(axis=1)
subset_df['avg_maximum_nights'] = subset_df['avg_maximum_nights'].astype(int)

# Confirming that the column creation has executed
subset_df['avg_maximum_nights']


0         1125
1         1125
2         1125
3         1125
4         1125
          ... 
395655     365
395656     365
395657     365
395658     365
395659     365
Name: avg_maximum_nights, Length: 395660, dtype: int32

In [46]:
'''
Exploring the columns 'bedrooms' and 'n_bedrooms'' to see whether 
they're duplicate of one another. If yes, one will be dropped. If not, both would be kept and
further clarification would be required to explain the difference between them
'''
# Checking if the two columns are duplicates of each other
bedroomcol_duplicate = subset_df['bedrooms'].equals(subset_df['n_bedrooms'])

# Printing the result
print("Are bedrooms and n_bedrooms columns duplicates?", bedroomcol_duplicate)

# The result shows that the columns are not duplicate

Are bedrooms and n_bedrooms columns duplicates? False


In [50]:
#Identifying instances in which the two columns dont have same values
# Filtering rows where values in host_listings_count and host_total_listings_count are different
diff_rows = subset_df[subset_df['n_bedrooms'] != subset_df['bedrooms']]
diff_rows[['n_bedrooms','bedrooms']]

# result shows that the difference is perhaps due to n_bedrooms containing string

,n_bedrooms,bedrooms
0,1 bedroom,1
1,1 bedroom,1
2,1 bedroom,1
3,1 bedroom,1
4,1 bedroom,1
...,...,...
395655,1 bed,1
395656,1 bed,1
395657,1 bed,1
395658,1 bed,1


In [52]:
# Checking out unique values for n_bedrooms
subset_df['n_bedrooms'].unique()

array(['1 bedroom', '2 bedrooms', '1 bed', '5 bedrooms', '3 beds',
       '2 beds', '3 bedrooms', '5 beds', '4 bedrooms', '7 bedrooms',
       'Studio', 'Half-bath', '4 beds', '9 beds', '6 bedrooms'],
      dtype=object)

In [53]:
# Checking out number of occurences for each unique value for n_bedrooms
subset_df['n_bedrooms'].value_counts()

1 bedroom     211335
2 bedrooms     83950
3 bedrooms     33215
1 bed          14600
Studio         12045
4 bedrooms     11315
2 beds          9490
5 bedrooms      8395
3 beds          6570
5 beds          1460
Half-bath       1460
4 beds           730
7 bedrooms       365
9 beds           365
6 bedrooms       365
Name: n_bedrooms, dtype: int64

In [54]:
# Checking out unique values for bedrooms
subset_df['bedrooms'].unique()

array([1, 2, 5, 3, 4, 6])

In [55]:
# Checking out number of occurences for each unique value for bedrooms
subset_df['bedrooms'].value_counts()

1    256960
2     80665
3     36865
4     12410
5      8395
6       365
Name: bedrooms, dtype: int64

In [58]:
'''
From the comparison above, n_bedrooms appears to be more descriptive than bedrooms column.
Recall, 'bedrooms' had nulls which were filled by median value. Perhaps the nulls could be
justified by studios and half-bath values in n_bedrooms. However, it will be difficult to distinguish 
whether the value of 1 in 'bedrooms' column is a filled null or an already existing value.
Both columns will be kept for possible data enrichment purposes, but they're likely to cause errors in
analysis

'''
# A quick observation
x = subset_df[subset_df['n_bedrooms'].isin(['Studio','Half-bath'])]
x[['n_bedrooms','bedrooms']]

,n_bedrooms,bedrooms
39420,Studio,1
39421,Studio,1
39422,Studio,1
39423,Studio,1
39424,Studio,1
...,...,...
392005,Half-bath,1
392006,Half-bath,1
392007,Half-bath,1
392008,Half-bath,1


In [95]:
subset_df.columns

Index(['listing_id', 'date', 'description', 'host_id', 'host_name',
       'host_since', 'host_location', 'host_response_time',
       'host_response_rate', 'host_acceptance_rate', 'host_is_superhost',
       'host_neighbourhood', 'host_listings_count',
       'host_total_listings_count', 'host_verifications',
       'host_has_profile_pic', 'host_identity_verified', 'property_type',
       'room_type', 'accommodates', 'bedrooms', 'beds', 'amenities', 'price_x',
       'price_y', 'adjusted_price', 'minimum_nights_x', 'minimum_nights_y',
       'maximum_nights_x', 'maximum_nights_y', 'has_availability',
       'number_of_reviews', 'review_scores_rating', 'review_scores_accuracy',
       'reviews_per_month', 'property_name', 'star_rating', 'n_bedrooms',
       'n_beds', 'n_baths', 'avg_minimum_nights', 'avg_maximum_nights'],
      dtype='object')

In [96]:
''' 
Dropping unwanted columns

Listing_name: 'property_name' which is a new column will replace it
price_x and price_y: 'adjusted_price' will be used instead
minimum_nights_x and minimum_nights_y: 'avg_minimum_nights' will be used instead
maximum_nights_x and maximum_nights_y: 'avg_maximum_nights' will be used instead

'''
# Dropping column
subset_df.drop(columns=['listing_name','price_x','price_y','minimum_nights_x','minimum_nights_y',
                       'maximum_nights_x','maximum_nights_y'], inplace=True)
# Confirming that the column no longer exists
subset_df.columns

Index(['listing_id', 'date', 'description', 'host_id', 'host_name',
       'host_since', 'host_location', 'host_response_time',
       'host_response_rate', 'host_acceptance_rate', 'host_is_superhost',
       'host_neighbourhood', 'host_listings_count',
       'host_total_listings_count', 'host_verifications',
       'host_has_profile_pic', 'host_identity_verified', 'property_type',
       'room_type', 'accommodates', 'bedrooms', 'beds', 'amenities',
       'adjusted_price', 'has_availability', 'number_of_reviews',
       'review_scores_rating', 'review_scores_accuracy', 'reviews_per_month',
       'property_name', 'star_rating', 'n_bedrooms', 'n_beds', 'n_baths',
       'avg_minimum_nights', 'avg_maximum_nights'],
      dtype='object')

In [115]:
# Reordering columns 
reordered_df = subset_df[['listing_id','property_name','property_type','description','star_rating',
                                  'bedrooms','n_bedrooms','beds','room_type','accommodates',
                                  'amenities','avg_minimum_nights','avg_maximum_nights','has_availability',
                                  'adjusted_price','number_of_reviews','reviews_per_month','review_scores_rating',
                                  'review_scores_accuracy','date','host_id','host_name','host_since',
                                  'host_location','host_response_time','host_response_rate','host_acceptance_rate',
                                  'host_is_superhost','host_neighbourhood','host_listings_count','host_total_listings_count',
                                  'host_verifications','host_has_profile_pic','host_identity_verified']].copy()

# Confirming that the reorder has worked
reordered_df.head(2)

,listing_id,property_name,property_type,description,star_rating,bedrooms,n_bedrooms,beds,room_type,accommodates,...,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_neighbourhood,host_listings_count,host_total_listings_count,host_verifications,host_has_profile_pic,host_identity_verified
0,54126420.0,Home in Albany,Entire home,1 bedroom private apartment located in a quiet...,4.80,1,1 bedroom,1,Entire home/apt,2.0,...,within an hour,100.0,98.0,t,Pine Hills,8,8,"['email', 'phone']",t,t
1,54126420.0,Home in Albany,Entire home,1 bedroom private apartment located in a quiet...,4.80,1,1 bedroom,1,Entire home/apt,2.0,...,within an hour,100.0,98.0,t,Pine Hills,8,8,"['email', 'phone']",t,t


In [125]:
# Double checking reordered_df for nulls 
reordered_df.isnull().sum() 

listing_id                   0
property_name                0
property_type                0
description                  0
star_rating                  0
bedrooms                     0
n_bedrooms                   0
beds                         0
room_type                    0
accommodates                 0
amenities                    0
avg_minimum_nights           0
avg_maximum_nights           0
has_availability             0
adjusted_price               0
number_of_reviews            0
reviews_per_month            0
review_scores_rating         0
review_scores_accuracy       0
date                         0
host_id                      0
host_name                    0
host_since                   0
host_location                0
host_response_time           0
host_response_rate           0
host_acceptance_rate         0
host_is_superhost            0
host_neighbourhood           0
host_listings_count          0
host_total_listings_count    0
host_verifications           0
host_has

In [126]:
# Double checking reordered_df for duplicates
reordered_df.duplicated().sum()

0

In [128]:
# Converting reordered_df to csv
reordered_df.to_csv('airbnb_albany_data.csv',index=False)